<a href="https://colab.research.google.com/github/maharshoaib786/Auto-Trading-Bot/blob/main/King_Queen_Grid_Bot_(v121_0_PreGen).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# king_queen_grid_bot.py  (v121.0-PreGen)
"""
An advanced grid bot for MetaTrader 5 featuring a dynamic "King/Queen" side management strategy.
This version pre-calculates the entire grid at startup and maintains a buffer of multiple pending orders.

Core Strategy:
The bot begins by placing a standard hedge (one BUY and one SELL). It then immediately pre-calculates
an entire grid of potential trade prices for both sides and stores them in memory. It aims to
maintain a constant number of open limit orders (e.g., 3) on each side. When a limit order
is filled, the bot places the next one from its in-memory list.

King/Queen Dynamic Strategy:
This is the bot's primary advanced feature, designed to manage a grid that has developed
a strong directional bias.

• Trigger Condition:
    - When the number of open trades on one side of a grid (e.g., BUYs) reaches the
      `KING_QUEEN_TRIGGER_COUNT`, the special mode is activated.

• Role Assignment:
    - The side that triggered the count becomes the "King Side" (the runner).
    - The opposite side becomes the "Queen Side" (the reactive hedger).
    - All position comments are updated to reflect this new status (e.g., "GridA_KingSide_gridbuy").

• King Side (Runner) Behavior:
    - Continues to place new pending grid orders from its pre-calculated list as the market
      moves in its favor.

• Queen Side (Hedger) Behavior:
    - Ceases its own grid-building activity (its list of grid levels is cleared).
    - It will only open a new position IF there are currently zero positions on the Queen side.
    - When it opens a trade, it's a single market order with special parameters:
        - Lot Size: It copies the lot size of the most recently opened King side position.
        - Stop Loss: Its SL is set to the Take Profit level of the most recent King side position.
        - Take Profit: It uses the standard `PROFIT_PIPS` for its own target.

• Grid Reset Condition:
    - Once all positions on the King side have been closed, the entire system resets.
    - The bot will close any remaining Queen side position, cancel all pending orders, and
      delete the grid's state from memory, ready for a new cycle.

Configuration is managed via a `.env` file.
"""
import os, sys, time, math, logging, traceback, datetime
import MetaTrader5 as mt5
from dotenv import load_dotenv

# Ensure UTF-8 encoding on Windows
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8")

load_dotenv()


# --- Configuration ---
SYMBOL                          = os.getenv("SYMBOL", "XAUUSDc")
LOT_SMALL                       = float(os.getenv("LOT_SMALL", 0.01))
LOT_MAX                         = float(os.getenv("LOT_MAX", 20))
GRID_LOT_SEQUENCE               = [0.01, 0.01, 0.02, 0.03, 0.03, 0.04, 0.06, 0.08, 0.10, 0.14, 0.18, 0.24, 0.31, 0.41, 0.55, 0.72, 0.95, 1.26, 1.67, 2.2, 2.91, 3.85, 5.09, 6.72, 8.89, 11.75, 15.53, 20.0]
GRID_STEP_PIPS                  = float(os.getenv("GRID_STEP_PIPS", 100))
DYNAMIC_STEP_PIPS               = float(os.getenv("DYNAMIC_STEP_PIPS", 100))
PROFIT_PIPS                     = float(os.getenv("PROFIT_PIPS", 350))
DYNAMIC_PROFIT_PIPS             = float(os.getenv("DYNAMIC_PROFIT_PIPS", 450))
DYNAMIC_POSITIONS_TRIGGER       = int(os.getenv("DYNAMIC_POSITIONS_TRIGGER", 27))
MAX_POSITIONS                   = int(os.getenv("MAX_POS", 60)) # Max total open positions across ALL grids
PROFIT_TARGET_AMT               = float(os.getenv("PROFIT_TARGET_AMT", 1000))
OPEN_DELAY                      = float(os.getenv("OPEN_DELAY", 2))
LOOP_MS                         = int(os.getenv("LOOP_MS", 100))
LOG_FILE                        = os.getenv("LOG_FILE", "king_queen_grid_bot.log")
LOGIN                           = int(os.getenv("MT5_LOGIN", 0))
PASSWORD                        = os.getenv("MT5_PASSWORD", "")
SERVER                          = os.getenv("MT5_SERVER", "")
PENDING_ORDER_EXPIRATION_MIN    = int(os.getenv("PENDING_ORDER_EXPIRATION_MIN", 0))
MAX_PENDING_ORDERS_AT_ONCE      = int(os.getenv("MAX_PENDING_ORDERS_AT_ONCE", 3))
LOG_BALANCE_INTERVAL            = int(os.getenv("LOG_BALANCE_INTERVAL", 100))
MAX_ACTIVE_GRIDS                = int(os.getenv("MAX_ACTIVE_GRIDS", 2)) # Limit for concurrent grids
INVALID_PRICE_RETRY_LIMIT       = int(os.getenv("INVALID_PRICE_RETRY_LIMIT", 10))
SLIPPAGE                        = int(os.getenv("SLIPPAGE", 30)) # Slippage for market orders
INVALID_PRICE_ADJUST_PIPS       = float(os.getenv("INVALID_PRICE_ADJUST_PIPS", 60.0)) # Pips to adjust price by on invalid price retry
ENABLE_TRADING_HOURS            = os.getenv("ENABLE_TRADING_HOURS", "False").lower() == "true"
TRADING_START_TIME_STR          = os.getenv("TRADING_START_TIME", "00:00") # Expected in HH:MM format, UTC
TRADING_END_TIME_STR            = os.getenv("TRADING_END_TIME", "23:59")   # Expected in HH:MM format, UTC
MAX_LOSS_AMT                    = float(os.getenv("MAX_LOSS_AMT", "0")) # Max allowable loss from initial_equity for current cycle. 0 or less disables.
KING_QUEEN_TRIGGER_COUNT        = int(os.getenv("KING_QUEEN_TRIGGER_COUNT", 10)) # Number of trades on one side to trigger King/Queen rename.

# --- Action Comment Strings & Search Keywords ---
ACTION_INITIAL_HEDGE_BUY = "initial_hedge_buy"
ACTION_INITIAL_HEDGE_SELL = "initial_hedge_sell"
ACTION_RE_HEDGE_BUY = "re_hedge_buy"
ACTION_RE_HEDGE_SELL = "re_hedge_sell"
ACTION_GRID_LIMIT_BUY = "gridbuy"
ACTION_GRID_LIMIT_SELL = "gridsell"
ACTION_CAPPED_BUY_TAG = "CappedBuy"
ACTION_CAPPED_SELL_TAG = "CappedSell"
ACTION_DYNAMIC_HEDGE_BUY_TAG = "DynamicHedgeBuy"
ACTION_DYNAMIC_HEDGE_SELL_TAG = "DynamicHedgeSell"
ACTION_SUFFIX_PROFIT_TARGET_CLOSE = "profit_target_close"
ACTION_SUFFIX_MAX_LOSS_CLOSE = "max_loss_limit_close"
SYSTEM_COMMENT_GRID_NAME = "System"
SEARCH_KEYWORD_CAPPED_GENERIC = "Capped"
SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC = "DynamicHedge"
SEARCH_KEYWORD_CAPPED_BUY_SPECIFIC = ACTION_CAPPED_BUY_TAG
SEARCH_KEYWORD_CAPPED_SELL_SPECIFIC = ACTION_CAPPED_SELL_TAG
SEARCH_KEYWORD_GRIDBUY_SANE = "gridbuy"
SEARCH_KEYWORD_GRIDSELL_SANE = "gridsell"
SEARCH_KEYWORD_MARKET_FALLBACK_SUBSTRING = "market_fallback"
SEARCH_KEYWORD_KINGSIDE = "KingSide"
SEARCH_KEYWORD_QUEENSIDE = "QueenSide"

# --- Logging setup ---
log = logging.getLogger("king_queen_grid_bot")
log.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s [%(levelname)s] %(message)s")
if not log.handlers:
    console_handler = logging.StreamHandler(sys.stdout)
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    log.addHandler(console_handler)
    file_handler = logging.FileHandler(LOG_FILE, mode="w", encoding="utf-8")
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)
    log.addHandler(file_handler)
log.propagate = False

# --- Parsed Configuration & Global State for Trading Hours ---
TRADING_START_TIME_OBJ: datetime.time | None = None
TRADING_END_TIME_OBJ: datetime.time | None = None
if ENABLE_TRADING_HOURS:
    try:
        TRADING_START_TIME_OBJ = datetime.datetime.strptime(TRADING_START_TIME_STR, "%H:%M").time()
        TRADING_END_TIME_OBJ = datetime.datetime.strptime(TRADING_END_TIME_STR, "%H:%M").time()
    except (ValueError, NameError):
        log.error(f"Invalid TRADING_START_TIME ('{TRADING_START_TIME_STR}') or TRADING_END_TIME ('{TRADING_END_TIME_STR}') format. Please use HH:MM. Trading hours feature disabled.")
        ENABLE_TRADING_HOURS = False

# --- State variables ---
initial_equity: float       = 0.0
grid_states = {}
base_magic_number = 12345
next_magic_number = 12346
loop_counter: int           = 0

# --- Trading Hours Utility ---
def is_trading_session_active() -> bool:
    """Checks if the current time is within the configured trading hours."""
    if not ENABLE_TRADING_HOURS or TRADING_START_TIME_OBJ is None or TRADING_END_TIME_OBJ is None:
        return True
    terminal_info_val = mt5.terminal_info()
    if terminal_info_val is None or not hasattr(terminal_info_val, 'time'):
        log.warning("is_trading_session_active: Could not get terminal info or server time. Assuming trading is active to be safe.")
        return True
    server_datetime_utc = datetime.datetime.fromtimestamp(terminal_info_val.time, tz=datetime.timezone.utc)
    current_time_utc = server_datetime_utc.time()
    if TRADING_START_TIME_OBJ <= TRADING_END_TIME_OBJ:
        is_active = TRADING_START_TIME_OBJ <= current_time_utc < TRADING_END_TIME_OBJ
    else: # Overnight case
        is_active = current_time_utc >= TRADING_START_TIME_OBJ or current_time_utc < TRADING_END_TIME_OBJ
    return is_active

# --- Utilities ---
def mt5_login():
    """Initializes MT5 connection and logs in."""
    global initial_equity
    log.info("Attempting to initialize MetaTrader 5...")
    if not mt5.initialize(login=LOGIN, password=PASSWORD, server=SERVER, timeout=10000):
        log.error(f"MT5 initialize() failed, error code = {mt5.last_error()}"); mt5.shutdown(); sys.exit(1)
    log.info(f"MetaTrader 5 initialized successfully. Terminal version: {mt5.version()}")
    account_info = mt5.account_info()
    if account_info is None:
        log.error(f"Failed to get account info, error code = {mt5.last_error()}"); mt5.shutdown(); sys.exit(1)
    log.info(f"Logged in to account: {account_info.login}, Server: {account_info.server}, Balance: {account_info.balance:.2f} {account_info.currency}")
    log.info(f"Selecting symbol: {SYMBOL}")
    if not mt5.symbol_select(SYMBOL, True):
        log.error(f"Symbol {SYMBOL} not found or not enabled in Market Watch, error code = {mt5.last_error()}"); mt5.shutdown(); sys.exit(1)
    symbol_info = mt5.symbol_info(SYMBOL)
    if symbol_info is None:
        log.error(f"Failed to get info for symbol {SYMBOL}, error code = {mt5.last_error()}"); mt5.shutdown(); sys.exit(1)
    log.info(f"Symbol {SYMBOL} selected. Description: {symbol_info.description}, Digits: {symbol_info.digits}")
    if initial_equity == 0.0:
        initial_equity = account_info.equity
        log.info(f"✅ Login successful. Account: {LOGIN}, Initial Equity for profit tracking set to: {initial_equity:.2f} {account_info.currency}")
    else:
        log.info(f"✅ Re-login successful. Account: {LOGIN}, Current Equity: {account_info.equity:.2f} {account_info.currency}. Profit tracking continues from {initial_equity:.2f}.")

def pip_val() -> float:
    """Determines the value of a single pip for the symbol."""
    info = mt5.symbol_info(SYMBOL)
    if not info:
        log.critical(f"CRITICAL: Could not get symbol info for {SYMBOL} in pip_val. Cannot determine pip value. Exiting.")
        raise ValueError(f"Failed to get symbol info for {SYMBOL}, cannot determine pip_val.")
    if info.digits in [5, 4]: return 0.0001
    if info.digits in [3, 2]: return 0.01
    log.warning(f"Uncommon number of digits ({info.digits}) for {SYMBOL}. Using point value as pip value: {info.point}")
    return info.point

def tp_price(direction: str, entry_price: float, pips_to_use: float) -> float:
    """Calculates the Take Profit price level."""
    price_offset = pips_to_use * pip_val()
    s_info_digits = mt5.symbol_info(SYMBOL).digits
    if direction == "BUY":
        return round(entry_price + price_offset, s_info_digits)
    else: # SELL
        return round(entry_price - price_offset, s_info_digits)

def adjust_lot(lot: float) -> float:
    """Adjusts a lot size to conform to the symbol's volume rules."""
    info = mt5.symbol_info(SYMBOL)
    if not info:
        log.error(f"Could not get symbol info for {SYMBOL} in adjust_lot. Returning original lot.")
        return lot
    volume_step = getattr(info, "volume_step", 0.01)
    volume_min = getattr(info, "volume_min", 0.01)
    volume_max = getattr(info, "volume_max", LOT_MAX)
    if volume_step <= 0: volume_step = 0.01
    lot = round(lot / volume_step) * volume_step
    lot = max(volume_min, lot)
    lot = min(volume_max, lot)
    lot = min(LOT_MAX, lot)
    precision = abs(int(math.log10(volume_step))) if 0 < volume_step < 1 else 0
    return round(lot, precision)

def format_mt5_comment(grid_name: str, action: str) -> str:
    """Formats a comment string to be MT5 compliant."""
    sane_action = "".join(filter(str.isalnum, action))
    if not sane_action: sane_action = "trade"
    comment = f"Grid{grid_name}_{sane_action}"
    return comment[:31]

# --- Trade execution (Market Orders) ---
def send_market_order(direction: str, lot: float, magic: int, grid_name: str, action_comment_str: str, stop_loss: float = 0.0, take_profit: float | None = None) -> mt5.OrderSendResult | None:
    """Sends a market order with retry logic for getting a valid price."""
    mt5_comment = format_mt5_comment(grid_name, action_comment_str)
    adjusted_lot = adjust_lot(lot)
    symbol_info_vol_min = getattr(mt5.symbol_info(SYMBOL), "volume_min", 0.01)
    if adjusted_lot < symbol_info_vol_min :
        log.warning(f"Market Order: Adjusted lot {adjusted_lot} is below minimum {symbol_info_vol_min} for {direction} {lot} ({mt5_comment}). Order not sent.")
        return None
    price, calculated_tp = 0.0, 0.0
    for attempt in range(5):
        tick = mt5.symbol_info_tick(SYMBOL)
        if tick and tick.bid > 0 and tick.ask > 0:
            price = tick.ask if direction == "BUY" else tick.bid
            if take_profit is not None:
                calculated_tp = take_profit
            else:
                positions = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]
                is_near_dynamic_trigger = (len(positions) + 1) >= DYNAMIC_POSITIONS_TRIGGER
                pips_to_use = DYNAMIC_PROFIT_PIPS if is_near_dynamic_trigger else PROFIT_PIPS
                calculated_tp = tp_price(direction, price, pips_to_use)
            break
        log.warning(f"Market Order: Attempt {attempt+1}/5: No valid tick for {direction} ({mt5_comment}). Retrying...")
        time.sleep(0.2)
    else:
        log.error(f"Market Order: Failed to get valid tick for {direction} ({mt5_comment}). Cannot send.")
        return None
    if price == 0.0:
        log.error(f"Market Order: Price for {SYMBOL} is zero. Cannot send {direction} ({mt5_comment}).")
        return None
    request = {
        "action": mt5.TRADE_ACTION_DEAL, "symbol": SYMBOL, "volume": adjusted_lot,
        "type": mt5.ORDER_TYPE_BUY if direction == "BUY" else mt5.ORDER_TYPE_SELL,
        "price": price, "tp": calculated_tp, "sl": stop_loss, "deviation": SLIPPAGE,
        "magic": magic, "comment": mt5_comment, "type_time": mt5.ORDER_TIME_GTC,
    }
    s_info = mt5.symbol_info(SYMBOL)
    if s_info and hasattr(s_info, 'filling_modes'):
        allowed_filling_types = s_info.filling_modes
        if mt5.ORDER_FILLING_IOC in allowed_filling_types: request["type_filling"] = mt5.ORDER_FILLING_IOC
        elif mt5.ORDER_FILLING_FOK in allowed_filling_types: request["type_filling"] = mt5.ORDER_FILLING_FOK
        elif len(allowed_filling_types) > 0: request["type_filling"] = allowed_filling_types[0]
    log.info(f"Sending Market Order: {direction} {adjusted_lot} {SYMBOL} @ Market (Ref: {price:.5f}) TP: {calculated_tp:.5f} SL: {stop_loss:.5f} Magic: {magic} Comment: {mt5_comment}")
    result = mt5.order_send(request)
    if result is None:
        log.error(f"Market Order send failed for {direction} {adjusted_lot} ({mt5_comment}). MT5 returned None. Error: {mt5.last_error()}")
        return None
    if result.retcode == mt5.TRADE_RETCODE_DONE:
        log.info(f"🟢 Market Order: {direction} {result.volume:.2f} @ {result.price:.5f} TP {calculated_tp:.5f} (Magic: {magic}, Ticket: {result.order}, Comment: {mt5_comment}) successfully placed.")
        return result
    else:
        log.error(f"🔴 Market Order send failed for {direction} {adjusted_lot} ({mt5_comment}). Retcode: {result.retcode}, MT5 Comment: {result.comment}, Error: {mt5.last_error()}")
        if result.retcode == 10026:
            log.critical("CRITICAL: Autotrading is disabled on the server. Please enable it in MT5 Terminal (Tools -> Options -> Expert Advisors -> Allow automated trading).")
        return None

# --- Trade execution (Limit Orders) ---
def place_limit_order(direction: str, price_level: float, lot: float, magic: int, grid_name: str, action_comment_str: str) -> mt5.OrderSendResult | None:
    """Places a limit order."""
    mt5_comment = format_mt5_comment(grid_name, action_comment_str)
    adjusted_lot = adjust_lot(lot)
    symbol_info_vol_min = getattr(mt5.symbol_info(SYMBOL), "volume_min", 0.01)
    if adjusted_lot < symbol_info_vol_min:
        log.warning(f"Limit Order: Adjusted lot {adjusted_lot} is below minimum {symbol_info_vol_min} for {direction} at {price_level:.5f}. Order not placed.")
        return None
    positions = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]
    is_near_dynamic_trigger = (len(positions) + 1) >= DYNAMIC_POSITIONS_TRIGGER
    pips_to_use = DYNAMIC_PROFIT_PIPS if is_near_dynamic_trigger else PROFIT_PIPS
    limit_order_tp = tp_price(direction, price_level, pips_to_use)
    order_type = mt5.ORDER_TYPE_BUY_LIMIT if direction == "BUY" else mt5.ORDER_TYPE_SELL_LIMIT
    request = {
        "action": mt5.TRADE_ACTION_PENDING, "symbol": SYMBOL, "volume": adjusted_lot,
        "type": order_type, "price": price_level, "tp": limit_order_tp, "sl": 0.0,
        "magic": magic, "comment": mt5_comment, "type_time": mt5.ORDER_TIME_GTC,
    }
    if PENDING_ORDER_EXPIRATION_MIN > 0:
        request["type_time"] = mt5.ORDER_TIME_SPECIFIED
        request["expiration"] = int(time.time()) + PENDING_ORDER_EXPIRATION_MIN * 60
    s_info = mt5.symbol_info(SYMBOL)
    if s_info and hasattr(s_info, 'filling_modes'):
        allowed_filling_types = s_info.filling_modes
        if mt5.ORDER_FILLING_RETURN in allowed_filling_types: request["type_filling"] = mt5.ORDER_FILLING_RETURN
        elif len(allowed_filling_types) > 0: request["type_filling"] = allowed_filling_types[0]
    log.info(f"Placing Limit Order: {direction} {adjusted_lot} {SYMBOL} @ {price_level:.5f} TP: {limit_order_tp:.5f} Magic: {magic} Comment: {mt5_comment}")
    result = mt5.order_send(request)
    if result is None:
        log.error(f"Limit Order placement failed (MT5 returned None) for {direction} {adjusted_lot} @ {price_level:.5f} ({mt5_comment}). Error: {mt5.last_error()}")
        return None
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        tick = mt5.symbol_info_tick(SYMBOL)
        log.error(f"🔴 Limit Order placement FAILED for {direction} {adjusted_lot} @ {price_level:.5f} ({mt5_comment}). Retcode: {result.retcode}, MT5 Comment: {result.comment}, Error: {mt5.last_error()}. Current Bid: {tick.bid if tick else 'N/A'}, Ask: {tick.ask if tick else 'N/A'}")
    else:
        log.info(f"🟢 Limit Order: {direction} {result.volume:.2f} {SYMBOL} @ {result.price:.5f} TP {limit_order_tp:.5f} (Magic: {magic}, Order Ticket: {result.order}, Comment: {mt5_comment}) successfully placed.")
    return result

def cancel_pending_orders_by_side(direction: str, magic: int | None = None):
    """Cancels all pending limit orders for a given side and magic number."""
    log.info(f"Attempting to cancel all pending {direction} limit orders for magic {magic if magic is not None else 'ALL'}.")
    order_type_to_cancel = mt5.ORDER_TYPE_BUY_LIMIT if direction == "BUY" else mt5.ORDER_TYPE_SELL_LIMIT
    pending_orders = mt5.orders_get(symbol=SYMBOL) or []
    cancelled_count = 0
    for order in pending_orders:
        if (magic is None or order.magic == magic) and order.type == order_type_to_cancel:
            del_request = {"action": mt5.TRADE_ACTION_REMOVE, "order": order.ticket, "symbol": SYMBOL}
            del_res = mt5.order_send(del_request)
            if del_res and del_res.retcode == mt5.TRADE_RETCODE_DONE:
                log.info(f"Cancelled pending {direction} limit order {order.ticket} for magic {order.magic}")
                cancelled_count += 1
            else:
                log.error(f"Failed to cancel pending {direction} limit order {order.ticket} (magic {order.magic}). Error: {mt5.last_error()}, Retcode: {del_res.retcode if del_res else 'N/A'}")
    if cancelled_count > 0:
        log.info(f"Cancelled {cancelled_count} pending {direction} limit orders.")
    else:
        log.info(f"No pending {direction} limit orders found for magic {magic if magic is not None else 'ALL'} to cancel.")

def sync_all_tps(direction: str, magic: int):
    """Syncs the TP of all positions on one side to match the TP of the newest position."""
    log.debug(f"Attempting to sync TPs for {direction} side for magic {magic}.")
    grid_state = grid_states.get(magic)
    if not grid_state:
        log.warning(f"sync_all_tps: Could not find state for magic {magic}. Skipping sync."); return
    if direction == "BUY" and grid_state.get('capped_buy', False):
        log.debug(f"TP Sync for BUY side (magic {magic}) skipped because this side is marked as capped in the grid state."); return
    if direction == "SELL" and grid_state.get('capped_sell', False):
        log.debug(f"TP Sync for SELL side (magic {magic}) skipped because this side is marked as capped in the grid state."); return
    side_to_sync = mt5.ORDER_TYPE_BUY if direction == "BUY" else mt5.ORDER_TYPE_SELL
    all_positions_of_side = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic and p.type == side_to_sync and SEARCH_KEYWORD_CAPPED_GENERIC not in p.comment and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]
    if not all_positions_of_side:
        log.debug(f"No open, non-capped, non-dynamic-hedge {direction} positions for magic {magic} to sync TPs for."); return
    try:
        all_positions_of_side.sort(key=lambda p: (p.time_msc if hasattr(p, 'time_msc') and p.time_msc > 0 else p.time, p.ticket), reverse=True)
        newest_position = all_positions_of_side[0]
    except IndexError:
        log.error(f"Could not determine newest position for {direction} side (magic {magic}) for TP sync (list was empty)."); return
    if newest_position.tp == 0:
        log.debug(f"Newest eligible {direction} position {newest_position.ticket} (magic {magic}) has no TP. Nothing to sync."); return
    new_tp_level = newest_position.tp
    log.debug(f"TP Sync check for {direction} (magic {magic}): Newest eligible position {newest_position.ticket} TP is {new_tp_level:.5f}. Comparing with others.")
    synced_count = 0
    mt5_comment_sync = format_mt5_comment(grid_states[magic]['name'], f"sync_tp_{newest_position.ticket}")
    for p in all_positions_of_side:
        if p.ticket == newest_position.ticket or math.isclose(p.tp, new_tp_level, abs_tol=mt5.symbol_info(SYMBOL).point * 0.1): continue
        log.info(f"TP Sync for {direction} (magic {magic}): Position {p.ticket} (TP: {p.tp:.5f}) will be synced to {new_tp_level:.5f}")
        modify_request = {"action": mt5.TRADE_ACTION_SLTP, "symbol": SYMBOL, "position": p.ticket, "tp": new_tp_level, "sl": p.sl, "magic": magic, "comment": mt5_comment_sync}
        result = mt5.order_send(modify_request)
        if result and result.retcode == mt5.TRADE_RETCODE_DONE:
            log.info(f"🔄 TP synced for {direction} position {p.ticket} to {new_tp_level:.5f}"); synced_count += 1
        else:
            log.warning(f"sync_all_tps: Failed to sync TP for {direction} position {p.ticket}. Error: {mt5.last_error()}, Retcode: {result.retcode if result else 'N/A'}")
    if synced_count > 0:
        log.info(f"✅ TP Sync Summary for {direction} (magic {magic}): {synced_count} positions updated to TP {new_tp_level:.5f}")
    else:
        log.debug(f"No TPs needed syncing for {direction} side (magic {magic}) based on newest eligible position {newest_position.ticket}.")

def handle_king_queen_rename(magic: int):
    """
    Checks if a grid side has reached KING_QUEEN_TRIGGER_COUNT positions.
    If so, renames the comments of all positions in that grid to reflect
    a "King" side (the one with many positions) and a "Queen" side.
    """
    grid_state = grid_states.get(magic)
    if not grid_state or grid_state.get('king_queen_side') or grid_state.get('capped_buy') or grid_state.get('capped_sell'): return
    positions = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]
    buy_positions = [p for p in positions if p.type == mt5.ORDER_TYPE_BUY and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]
    sell_positions = [p for p in positions if p.type == mt5.ORDER_TYPE_SELL and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]
    king_side_direction = None
    if len(buy_positions) >= KING_QUEEN_TRIGGER_COUNT: king_side_direction = "BUY"
    elif len(sell_positions) >= KING_QUEEN_TRIGGER_COUNT: king_side_direction = "SELL"
    if not king_side_direction: return
    log.warning(f"👑 KING/QUEEN TRIGGER HIT for grid {grid_state['name']} (magic {magic}). {king_side_direction} side is now King Side.")
    grid_state['king_queen_side'] = king_side_direction
    (king_positions, old_queen_positions) = (buy_positions, sell_positions) if king_side_direction == "BUY" else (sell_positions, buy_positions)
    king_tag = SEARCH_KEYWORD_KINGSIDE
    queen_side_str = "SELL" if king_side_direction == "BUY" else "BUY"

    # --- NEW: Clear the Queen side's grid levels from memory ---
    if queen_side_str == "BUY":
        grid_state['buy_grid_levels'] = []
        grid_state['buy_levels_placed'] = 0
    else:
        grid_state['sell_grid_levels'] = []
        grid_state['sell_levels_placed'] = 0
    log.info(f"K/Q Trigger: Cleared pending grid levels for the new Queen side ({queen_side_str}).")

    def _rename_king_positions(positions_to_rename: list, tag: str):
        renamed_count = 0
        for p in positions_to_rename:
            if tag in p.comment or SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC in p.comment: continue
            try:
                parts = p.comment.split('_', 1)
                new_comment = f"{parts[0]}_{tag}_{parts[1]}"
                new_comment = new_comment[:31]
            except IndexError:
                log.warning(f"Could not parse comment '{p.comment}' for renaming. Skipping."); continue
            log.info(f"Renaming position {p.ticket} comment from '{p.comment}' to '{new_comment}'")
            modify_request = {"action": mt5.TRADE_ACTION_SLTP, "position": p.ticket, "tp": p.tp, "sl": p.sl, "magic": magic, "comment": new_comment}
            result = mt5.order_send(modify_request)
            if result and result.retcode == mt5.TRADE_RETCODE_DONE: renamed_count += 1
            else: log.warning(f"Failed to rename position {p.ticket}. Retcode: {result.retcode if result else 'N/A'}, Error: {mt5.last_error()}")
        if renamed_count > 0: log.info(f"✅ Renamed {renamed_count} positions to {tag}.")

    _rename_king_positions(king_positions, king_tag)
    log.info(f"K/Q Trigger: Closing {len(old_queen_positions)} positions on the new Queen side ({queen_side_str}) to make way for the hedger.")
    for p in old_queen_positions:
        close_direction = mt5.ORDER_TYPE_SELL if p.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
        tick = mt5.symbol_info_tick(SYMBOL)
        if not tick: log.error(f"K/Q Trigger: Could not get tick to close Queen position {p.ticket}. Skipping."); continue
        close_price = tick.bid if p.type == mt5.ORDER_TYPE_BUY else tick.ask
        close_request = { "action": mt5.TRADE_ACTION_DEAL, "symbol": SYMBOL, "volume": p.volume, "type": close_direction, "position": p.ticket, "price": close_price, "deviation": SLIPPAGE, "magic": magic, "comment": format_mt5_comment(grid_state['name'], f"kq_clear_{queen_side_str.lower()}")}
        result = mt5.order_send(close_request)
        if result and result.retcode == mt5.TRADE_RETCODE_DONE: log.info(f"K/Q Trigger: Closed Queen position {p.ticket}.")
        else: log.warning(f"K/Q Trigger: Failed to close Queen position {p.ticket}. Retcode: {result.retcode if result else 'N/A'}")
    cancel_pending_orders_by_side(queen_side_str, magic)
    if king_positions:
        king_positions.sort(key=lambda p: (p.time_msc if hasattr(p, 'time_msc') and p.time_msc > 0 else p.time, p.ticket), reverse=True)
        last_king_pos = king_positions[0]
        lot_from_king = last_king_pos.volume
        sl_from_king_tp = last_king_pos.tp
        if sl_from_king_tp > 0:
            action_comment = f"{SEARCH_KEYWORD_QUEENSIDE}_{queen_side_str.lower()}_init"
            log.info(f"K/Q Trigger: Instantly opening Queen ({queen_side_str}) hedger position for magic {magic}: Lot {lot_from_king}, SL based on King's TP {sl_from_king_tp:.5f}")
            send_market_order(direction=queen_side_str, lot=lot_from_king, magic=magic, grid_name=grid_state['name'], action_comment_str=action_comment, stop_loss=sl_from_king_tp, take_profit=None)
        else:
            log.warning(f"K/Q Trigger: Cannot open instant Queen position for magic {magic}. Last King position {last_king_pos.ticket} has no TP set.")
    else:
        log.warning(f"K/Q Trigger: Cannot open instant Queen position for magic {magic} because no King side positions were found.")

def sync_queen_sl_with_king_tp(magic: int):
    """
    In King/Queen mode, this function continuously syncs the Stop Loss of the
    Queen side position to match the Take Profit of the newest King side position.
    """
    grid_state = grid_states.get(magic)
    if not grid_state or not grid_state.get('king_queen_side'): return
    king_side_str = grid_state['king_queen_side']
    queen_side_str = "SELL" if king_side_str == "BUY" else "BUY"
    log.debug(f"Syncing Queen SL for magic {magic}. King is {king_side_str}, Queen is {queen_side_str}.")
    positions = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]
    queen_side_type = mt5.ORDER_TYPE_BUY if queen_side_str == "BUY" else mt5.ORDER_TYPE_SELL
    queen_positions = [p for p in positions if p.type == queen_side_type]
    if not queen_positions:
        log.debug(f"No Queen ({queen_side_str}) position found for magic {magic} to sync SL for."); return
    if len(queen_positions) > 1:
        log.warning(f"Expected 1 Queen ({queen_side_str}) position for magic {magic}, but found {len(queen_positions)}. Skipping SL sync."); return
    queen_position = queen_positions[0]
    king_side_type = mt5.ORDER_TYPE_BUY if king_side_str == "BUY" else mt5.ORDER_TYPE_SELL
    king_positions = [p for p in positions if p.type == king_side_type and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]
    if not king_positions:
        log.debug(f"No King ({king_side_str}) positions found for magic {magic}. Cannot sync Queen's SL."); return
    king_positions.sort(key=lambda p: (p.time_msc if hasattr(p, 'time_msc') and p.time_msc > 0 else p.time, p.ticket), reverse=True)
    newest_king_position = king_positions[0]
    new_sl_for_queen = newest_king_position.tp
    if new_sl_for_queen == 0.0:
        log.debug(f"Newest King position {newest_king_position.ticket} has no TP. Cannot sync Queen's SL."); return
    if math.isclose(queen_position.sl, new_sl_for_queen, abs_tol=mt5.symbol_info(SYMBOL).point * 0.1): return
    log.info(f"🔄 Syncing Queen position {queen_position.ticket} SL from {queen_position.sl:.5f} to King's newest TP {new_sl_for_queen:.5f} (from King pos {newest_king_position.ticket}).")
    mt5_comment_sync = format_mt5_comment(grid_state['name'], f"sync_q_sl_{newest_king_position.ticket}")
    modify_request = {"action": mt5.TRADE_ACTION_SLTP, "position": queen_position.ticket, "sl": new_sl_for_queen, "tp": queen_position.tp, "magic": magic, "comment": mt5_comment_sync}
    result = mt5.order_send(modify_request)
    if result and result.retcode == mt5.TRADE_RETCODE_DONE:
        log.info(f"✅ Queen SL for position {queen_position.ticket} successfully synced to {new_sl_for_queen:.5f}.")
    else:
        log.warning(f"Failed to sync Queen SL for position {queen_position.ticket}. Error: {mt5.last_error()}, Retcode: {result.retcode if result else 'N/A'}")

def close_all_symbol_positions(reason_comment_suffix: str):
    """Closes all open positions for the configured SYMBOL."""
    log.info(f"Attempting to close all positions for {SYMBOL} due to: {reason_comment_suffix}")
    open_positions = mt5.positions_get(symbol=SYMBOL) or []
    closed_count = 0
    if not open_positions:
        log.info(f"No open positions found for {SYMBOL} to close for {reason_comment_suffix}."); return
    for position in open_positions:
        close_direction = mt5.ORDER_TYPE_SELL if position.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
        tick = mt5.symbol_info_tick(SYMBOL)
        if not tick or tick.bid == 0 or tick.ask == 0:
            log.error(f"Could not get valid tick to close position {position.ticket} for {reason_comment_suffix}. Skipping."); continue
        close_price = tick.bid if position.type == mt5.ORDER_TYPE_BUY else tick.ask
        close_request = {"action": mt5.TRADE_ACTION_DEAL, "symbol": SYMBOL, "volume": position.volume, "type": close_direction, "position": position.ticket, "price": close_price, "deviation": SLIPPAGE + 20, "magic": position.magic, "comment": format_mt5_comment(SYSTEM_COMMENT_GRID_NAME, reason_comment_suffix)}
        s_info = mt5.symbol_info(SYMBOL)
        if s_info and hasattr(s_info, 'filling_modes'):
            allowed = s_info.filling_modes
            if mt5.ORDER_FILLING_IOC in allowed: close_request["type_filling"] = mt5.ORDER_FILLING_IOC
            elif mt5.ORDER_FILLING_FOK in allowed: close_request["type_filling"] = mt5.ORDER_FILLING_FOK
            elif len(allowed) > 0: close_request["type_filling"] = allowed[0]
        result = mt5.order_send(close_request)
        if result and result.retcode == mt5.TRADE_RETCODE_DONE:
            log.info(f"Closed position {position.ticket} (Vol: {position.volume}, Type: {'BUY' if position.type == mt5.ORDER_TYPE_BUY else 'SELL'}) for {reason_comment_suffix}."); closed_count +=1
        else:
            log.error(f"Failed to close position {position.ticket} for {reason_comment_suffix}. Error: {mt5.last_error()}, Retcode: {result.retcode if result else 'N/A'}")
            if result and result.retcode == 10026: log.critical("CRITICAL: Autotrading is disabled on the server.")
    log.info(f"Closed {closed_count}/{len(open_positions)} open positions for {SYMBOL} due to {reason_comment_suffix}.")

def generate_grid_levels(start_price: float, direction: str) -> list[float]:
    """Pre-calculates a full list of grid price levels."""
    levels = []
    symbol_info_obj = mt5.symbol_info(SYMBOL)
    if not symbol_info_obj:
        log.error(f"generate_grid_levels: Could not get symbol info for {SYMBOL}. Cannot generate levels.")
        return []

    s_info_digits = symbol_info_obj.digits
    current_price = start_price

    for i in range(len(GRID_LOT_SEQUENCE)):
        step_pips = GRID_STEP_PIPS
        price_offset = step_pips * pip_val()

        if direction == "BUY":
            next_price = round(current_price - price_offset, s_info_digits)
        else: # SELL
            next_price = round(current_price + price_offset, s_info_digits)

        levels.append(next_price)
        current_price = next_price

    log.info(f"Generated {len(levels)} grid levels for {direction} side, starting from {start_price:.{s_info_digits}f}.")
    return levels

def hedge_if_empty():
    """Places the initial BUY and SELL hedge and generates the first grid."""
    global grid_states
    if grid_states or (mt5.positions_get(symbol=SYMBOL) or []):
        return

    if ENABLE_TRADING_HOURS and not is_trading_session_active():
        if loop_counter % (LOG_BALANCE_INTERVAL * 10) == 1 :
            log.info(f"hedge_if_empty: Outside trading hours ({TRADING_START_TIME_STR} - {TRADING_END_TIME_STR} UTC). Initial hedge not placed.")
        return

    log.info("🔄 No active grids and no positions found. Initiating initial grid (Magic: %d)...", base_magic_number)
    buy_result = send_market_order("BUY", LOT_SMALL, base_magic_number, "A", ACTION_INITIAL_HEDGE_BUY)
    sell_result = send_market_order("SELL", LOT_SMALL, base_magic_number, "A", ACTION_INITIAL_HEDGE_SELL)

    if buy_result and sell_result:
        buy_open_price = buy_result.price
        sell_open_price = sell_result.price

        buy_levels = generate_grid_levels(buy_open_price, "BUY")
        sell_levels = generate_grid_levels(sell_open_price, "SELL")

        grid_states[base_magic_number] = {
            "name": "A",
            "buy_sequence_index": 0, "sell_sequence_index": 0,
            "prev_buy_count": 1, "prev_sell_count": 1,
            "capped_buy": False, "capped_sell": False,
            "king_queen_side": None,
            "buy_grid_levels": buy_levels,
            "sell_grid_levels": sell_levels,
            "buy_levels_placed": 0,
            "sell_levels_placed": 0,
        }
        log.info("Initial market hedge orders placed successfully for Grid A (magic %d).", base_magic_number)
        time.sleep(OPEN_DELAY)
    else:
        log.error("Failed to place one or both initial market hedge orders.")

def handle_closed_hedge(magic: int):
    """
    Detects when positions close and manages the grid state accordingly.
    """
    global grid_states
    grid_state = grid_states.get(magic)
    if not grid_state: return

    positions_in_grid = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]

    # --- King/Queen Reset Logic ---
    king_queen_direction = grid_state.get('king_queen_side')
    if king_queen_direction:
        king_side_type = mt5.ORDER_TYPE_BUY if king_queen_direction == "BUY" else mt5.ORDER_TYPE_SELL
        current_king_positions = [p for p in positions_in_grid if p.type == king_side_type and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]

        if not current_king_positions:
            log.warning(f"👑 King side ({king_queen_direction}) for grid {grid_state['name']} has closed. Resetting grid and closing Queen side.")
            cancel_pending_orders_by_side("BUY", magic)
            cancel_pending_orders_by_side("SELL", magic)
            positions_to_close = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]
            for p in positions_to_close:
                close_direction = mt5.ORDER_TYPE_SELL if p.type == mt5.ORDER_TYPE_BUY else mt5.ORDER_TYPE_BUY
                tick = mt5.symbol_info_tick(SYMBOL)
                if not tick: log.error(f"Could not get tick to close remaining position {p.ticket} during King/Queen reset. Skipping."); continue
                close_price = tick.bid if p.type == mt5.ORDER_TYPE_BUY else tick.ask
                close_request = { "action": mt5.TRADE_ACTION_DEAL, "symbol": SYMBOL, "volume": p.volume, "type": close_direction, "position": p.ticket, "price": close_price, "deviation": SLIPPAGE, "magic": magic, "comment": format_mt5_comment(grid_state['name'], "kq_reset_close")}
                mt5.order_send(close_request)
            if magic in grid_states:
                del grid_states[magic]
                log.info(f"Grid state for magic {magic} removed due to King/Queen reset. A new hedge will be placed if needed.")
            return

        grid_state["prev_buy_count"] = sum(1 for p in positions_in_grid if p.type == mt5.ORDER_TYPE_BUY and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment)
        grid_state["prev_sell_count"] = sum(1 for p in positions_in_grid if p.type == mt5.ORDER_TYPE_SELL and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment)
        return

    # --- Normal Mode Logic ---
    current_buy_count = sum(1 for p in positions_in_grid if p.type == mt5.ORDER_TYPE_BUY and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment)
    current_sell_count = sum(1 for p in positions_in_grid if p.type == mt5.ORDER_TYPE_SELL and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment)

    if current_buy_count < grid_state.get("prev_buy_count", current_buy_count + 1):
        log.info(f"🔔 BUY position closed in grid {grid_state['name']} (magic {magic}). Current non-hedge BUY count: {current_buy_count}.")
        cancel_pending_orders_by_side("BUY", magic)
        if not grid_state.get('capped_buy'):
            grid_state["buy_sequence_index"] = 0
            if current_buy_count == 0 and current_sell_count > 0:
                if not (ENABLE_TRADING_HOURS and not is_trading_session_active()):
                    log.info(f"Grid {grid_state['name']} BUY side empty, SELL side active. Re-hedging BUY and regenerating levels.")
                    buy_result = send_market_order("BUY", LOT_SMALL, magic, grid_state['name'], action_comment_str=ACTION_RE_HEDGE_BUY)
                    if buy_result:
                        grid_state['buy_grid_levels'] = generate_grid_levels(buy_result.price, "BUY")
                        grid_state['buy_levels_placed'] = 0
                        time.sleep(OPEN_DELAY)
                else:
                    log.info(f"Grid {grid_state['name']} BUY side re-hedge skipped: Outside trading hours.")

    if current_sell_count < grid_state.get("prev_sell_count", current_sell_count + 1):
        log.info(f"🔔 SELL position closed in grid {grid_state['name']} (magic {magic}). Current non-hedge SELL count: {current_sell_count}.")
        cancel_pending_orders_by_side("SELL", magic)
        if not grid_state.get('capped_sell'):
            grid_state["sell_sequence_index"] = 0
            if current_sell_count == 0 and current_buy_count > 0:
                if not (ENABLE_TRADING_HOURS and not is_trading_session_active()):
                    log.info(f"Grid {grid_state['name']} SELL side empty, BUY side active. Re-hedging SELL and regenerating levels.")
                    sell_result = send_market_order("SELL", LOT_SMALL, magic, grid_state['name'], action_comment_str=ACTION_RE_HEDGE_SELL)
                    if sell_result:
                        grid_state['sell_grid_levels'] = generate_grid_levels(sell_result.price, "SELL")
                        grid_state['sell_levels_placed'] = 0
                        time.sleep(OPEN_DELAY)
                else:
                    log.info(f"Grid {grid_state['name']} SELL side re-hedge skipped: Outside trading hours.")

    grid_state["prev_buy_count"] = current_buy_count
    grid_state["prev_sell_count"] = current_sell_count

    if not positions_in_grid:
        log.warning(f"Grid {grid_state['name']} (magic {magic}) is now completely empty. Deactivating this grid.")
        cancel_pending_orders_by_side("BUY", magic)
        cancel_pending_orders_by_side("SELL", magic)
        if magic in grid_states:
            del grid_states[magic]
            log.info(f"Grid state for magic {magic} removed.")

def manage_pending_orders(magic: int):
    """
    Ensures the desired number of pending limit orders are always active by
    pulling pre-calculated prices from the in-memory grid level lists.
    """
    global grid_states
    grid_state = grid_states.get(magic)
    if not grid_state: return

    if len(mt5.positions_get(symbol=SYMBOL) or []) >= MAX_POSITIONS:
        log.warning(f"Max open positions ({MAX_POSITIONS}) reached. No new grid limit orders will be placed for magic {magic}.")
        return

    if ENABLE_TRADING_HOURS and not is_trading_session_active():
        if loop_counter % (LOG_BALANCE_INTERVAL * 10) == 1 :
            log.info(f"manage_pending_orders (Magic {magic}): Outside trading hours. No new limit orders will be placed.")
        return

    s_info = mt5.symbol_info(SYMBOL)
    if not s_info: return

    pending_orders = [o for o in (mt5.orders_get(symbol=SYMBOL) or []) if o.magic == magic]

    # --- MANAGE BUY LIMITS ---
    is_buy_side_active = not grid_state.get('capped_buy', False) and grid_state.get('king_queen_side') != 'SELL'

    if is_buy_side_active:
        current_pending_buy_limits = sum(1 for o in pending_orders if o.type == mt5.ORDER_TYPE_BUY_LIMIT)
        while current_pending_buy_limits < MAX_PENDING_ORDERS_AT_ONCE:
            placed_idx = grid_state.get("buy_levels_placed", 0)
            buy_levels = grid_state.get("buy_grid_levels", [])
            if placed_idx >= len(buy_levels):
                log.debug(f"No more pre-calculated BUY levels to place for magic {magic}.")
                break
            price_level = buy_levels[placed_idx]
            lot_idx = grid_state.get("buy_sequence_index", 0) + placed_idx
            raw_lot = GRID_LOT_SEQUENCE[lot_idx] if lot_idx < len(GRID_LOT_SEQUENCE) else GRID_LOT_SEQUENCE[-1]
            current_lot = adjust_lot(raw_lot)
            action_comment = f"{SEARCH_KEYWORD_KINGSIDE}_{ACTION_GRID_LIMIT_BUY}" if grid_state.get('king_queen_side') == 'BUY' else ACTION_GRID_LIMIT_BUY

            order_result = place_limit_order("BUY", price_level, current_lot, magic, grid_state['name'], action_comment)
            if order_result and order_result.retcode == mt5.TRADE_RETCODE_DONE:
                grid_state["buy_levels_placed"] += 1
                current_pending_buy_limits += 1
                time.sleep(0.1)
            else:
                log.warning(f"Failed to place BUY limit order for magic {magic} at level {price_level}. Stopping for this cycle.")
                break

    # --- MANAGE SELL LIMITS ---
    is_sell_side_active = not grid_state.get('capped_sell', False) and grid_state.get('king_queen_side') != 'BUY'

    if is_sell_side_active:
        current_pending_sell_limits = sum(1 for o in pending_orders if o.type == mt5.ORDER_TYPE_SELL_LIMIT)
        while current_pending_sell_limits < MAX_PENDING_ORDERS_AT_ONCE:
            placed_idx = grid_state.get("sell_levels_placed", 0)
            sell_levels = grid_state.get("sell_grid_levels", [])
            if placed_idx >= len(sell_levels):
                log.debug(f"No more pre-calculated SELL levels to place for magic {magic}.")
                break
            price_level = sell_levels[placed_idx]
            lot_idx = grid_state.get("sell_sequence_index", 0) + placed_idx
            raw_lot = GRID_LOT_SEQUENCE[lot_idx] if lot_idx < len(GRID_LOT_SEQUENCE) else GRID_LOT_SEQUENCE[-1]
            current_lot = adjust_lot(raw_lot)
            action_comment = f"{SEARCH_KEYWORD_KINGSIDE}_{ACTION_GRID_LIMIT_SELL}" if grid_state.get('king_queen_side') == 'SELL' else ACTION_GRID_LIMIT_SELL

            order_result = place_limit_order("SELL", price_level, current_lot, magic, grid_state['name'], action_comment)
            if order_result and order_result.retcode == mt5.TRADE_RETCODE_DONE:
                grid_state["sell_levels_placed"] += 1
                current_pending_sell_limits += 1
                time.sleep(0.1)
            else:
                log.warning(f"Failed to place SELL limit order for magic {magic} at level {price_level}. Stopping for this cycle.")
                break

def manage_king_queen_sides(magic: int):
    """Manages a grid that is in the King/Queen state, focusing on the Queen side."""
    grid_state = grid_states.get(magic)
    if not grid_state or not grid_state.get('king_queen_side'): return
    if ENABLE_TRADING_HOURS and not is_trading_session_active(): return

    king_side_str = grid_state['king_queen_side']
    queen_side_str = "SELL" if king_side_str == "BUY" else "BUY"
    positions = [p for p in (mt5.positions_get(symbol=SYMBOL) or []) if p.magic == magic]

    queen_side_type = mt5.ORDER_TYPE_BUY if queen_side_str == "BUY" else mt5.ORDER_TYPE_SELL
    queen_positions = [p for p in positions if p.type == queen_side_type]

    if not queen_positions:
        log.info(f"Queen side ({queen_side_str}) for magic {magic} is empty. Attempting to open a new Queen position.")
        king_side_type = mt5.ORDER_TYPE_BUY if king_side_str == "BUY" else mt5.ORDER_TYPE_SELL
        king_positions = [p for p in positions if p.type == king_side_type and SEARCH_KEYWORD_DYNAMIC_HEDGE_GENERIC not in p.comment]
        if king_positions:
            king_positions.sort(key=lambda p: (p.time_msc if hasattr(p, 'time_msc') and p.time_msc > 0 else p.time, p.ticket), reverse=True)
            last_king_pos = king_positions[0]
            lot_from_king = last_king_pos.volume
            sl_from_king_tp = last_king_pos.tp
            if sl_from_king_tp > 0:
                action_comment = f"{SEARCH_KEYWORD_QUEENSIDE}_{queen_side_str.lower()}"
                log.info(f"Opening Queen ({queen_side_str}) position for magic {magic}: Lot {lot_from_king}, SL based on King's TP {sl_from_king_tp:.5f}")
                send_market_order(direction=queen_side_str, lot=lot_from_king, magic=magic, grid_name=grid_state['name'], action_comment_str=action_comment, stop_loss=sl_from_king_tp, take_profit=None)
            else:
                log.warning(f"Cannot open Queen position for magic {magic}. Last King position {last_king_pos.ticket} has no TP set.")
        else:
            log.warning(f"Cannot open Queen position for magic {magic} because no King side positions were found.")

def run():
    """The main execution loop of the bot."""
    global initial_equity, loop_counter, grid_states, next_magic_number

    mt5_login()
    if ENABLE_TRADING_HOURS:
        log.info(f"TRADING HOURS ENABLED: {TRADING_START_TIME_STR} - {TRADING_END_TIME_STR} UTC" if TRADING_START_TIME_OBJ and TRADING_END_TIME_OBJ else "TRADING HOURS DISABLED due to parsing error.")

    existing_positions = mt5.positions_get(symbol=SYMBOL) or []
    if existing_positions:
        magics = sorted(list(set(p.magic for p in existing_positions)))
        log.info(f"↻ Restarted. Found existing positions with magic numbers: {magics}")
        if magics:
            current_max_magic = max(magics)
            if current_max_magic >= base_magic_number:
                next_magic_number = current_max_magic + 1
        for i, magic in enumerate(magics):
            grid_name = chr(ord('A') + i % 26)
            positions_in_grid = [p for p in existing_positions if p.magic == magic]
            buy_positions = [p for p in positions_in_grid if p.type == mt5.ORDER_TYPE_BUY]
            sell_positions = [p for p in positions_in_grid if p.type == mt5.ORDER_TYPE_SELL]
            oldest_buy = min(buy_positions, key=lambda p: p.price_open) if buy_positions else None
            oldest_sell = max(sell_positions, key=lambda p: p.price_open) if sell_positions else None
            buy_levels = generate_grid_levels(oldest_buy.price_open, "BUY") if oldest_buy else []
            sell_levels = generate_grid_levels(oldest_sell.price_open, "SELL") if oldest_sell else []

            grid_states[magic] = {
                "name": grid_name, "buy_sequence_index": len(buy_positions) -1 if buy_positions else 0,
                "sell_sequence_index": len(sell_positions) -1 if sell_positions else 0,
                "prev_buy_count": len(buy_positions), "prev_sell_count": len(sell_positions),
                "capped_buy": any(SEARCH_KEYWORD_CAPPED_BUY_SPECIFIC in p.comment for p in buy_positions),
                "capped_sell": any(SEARCH_KEYWORD_CAPPED_SELL_SPECIFIC in p.comment for p in sell_positions),
                "king_queen_side": "BUY" if any(SEARCH_KEYWORD_KINGSIDE in p.comment for p in buy_positions) else ("SELL" if any(SEARCH_KEYWORD_KINGSIDE in p.comment for p in sell_positions) else None),
                "buy_grid_levels": buy_levels, "sell_grid_levels": sell_levels,
                "buy_levels_placed": 0, "sell_levels_placed": 0
            }
            log.info(f"Grid {grid_name} (magic {magic}) reconstructed with new grid levels.")
    else:
        hedge_if_empty()

    try:
        while True:
            loop_counter += 1
            if mt5.terminal_info() is None:
                log.error("Lost connection to MT5 terminal. Attempting to reconnect...")
                mt5.shutdown(); time.sleep(10); mt5_login()
                if mt5.terminal_info() is None:
                    log.critical("Failed to reconnect. Exiting."); break

            if loop_counter % LOG_BALANCE_INTERVAL == 0:
                acc_info = mt5.account_info()
                if acc_info: log.info(f"Account Status - Balance: {acc_info.balance:.2f}, Equity: {acc_info.equity:.2f}")

            account_info = mt5.account_info()
            if account_info:
                if PROFIT_TARGET_AMT > 0 and (account_info.equity - initial_equity) >= PROFIT_TARGET_AMT:
                    log.warning(f"🎯 PROFIT TARGET of {PROFIT_TARGET_AMT:.2f} REACHED!")
                    close_all_symbol_positions(ACTION_SUFFIX_PROFIT_TARGET_CLOSE)
                    log.info("Resetting bot state for new cycle...")
                    grid_states = {}
                    new_acc_info = mt5.account_info()
                    initial_equity = new_acc_info.equity if new_acc_info else account_info.equity
                    log.info(f"New initial_equity for profit tracking: {initial_equity:.2f}")
                    hedge_if_empty()
                    continue
                if MAX_LOSS_AMT > 0 and (initial_equity - account_info.equity) >= MAX_LOSS_AMT:
                    log.critical(f"☠️ MAXIMUM LOSS LIMIT of {MAX_LOSS_AMT:.2f} REACHED!")
                    close_all_symbol_positions(ACTION_SUFFIX_MAX_LOSS_CLOSE)
                    sys.exit(1)

            for magic_key in list(grid_states.keys()):
                if magic_key not in grid_states: continue
                handle_closed_hedge(magic_key)
                if magic_key not in grid_states: continue

                grid_state = grid_states[magic_key]
                if grid_state.get('king_queen_side'):
                    manage_king_queen_sides(magic_key)
                    sync_queen_sl_with_king_tp(magic_key)

                manage_pending_orders(magic_key)

                sync_all_tps("BUY", magic_key)
                sync_all_tps("SELL", magic_key)
                handle_king_queen_rename(magic_key)

            hedge_if_empty()
            time.sleep(LOOP_MS / 1000.0)

    except KeyboardInterrupt:
        log.info("User requested shutdown (KeyboardInterrupt).")
    except Exception as e:
        log.error(f"An unexpected error occurred in the main loop: {e}")
        log.error(traceback.format_exc())
    finally:
        log.info("Shutting down MetaTrader 5 connection...")
        mt5.shutdown()
        log.info("Bot stopped.")

if __name__ == '__main__':
    log.info(f"Starting King/Queen Grid Bot (v121.0-PreGen)")
    log.info(f"SYMBOL: {SYMBOL}, LOT_SMALL: {LOT_SMALL}, MAX_LOT: {LOT_MAX}")
    log.info(f"GRID_PIPS: {GRID_STEP_PIPS}, PROFIT_PIPS: {PROFIT_PIPS}")
    log.info(f"MAX_POS: {MAX_POSITIONS}, PROFIT_TARGET: {PROFIT_TARGET_AMT}")
    log.info(f"MAX_LOSS_AMT: {MAX_LOSS_AMT if MAX_LOSS_AMT > 0 else 'Disabled'}")
    log.info(f"KING/QUEEN TRIGGER COUNT: {KING_QUEEN_TRIGGER_COUNT}")
    log.info(f"MAX PENDING ORDERS AT ONCE: {MAX_PENDING_ORDERS_AT_ONCE}")
    log.info(f"Configured Trading Hours: Enabled={ENABLE_TRADING_HOURS}, Start='{TRADING_START_TIME_STR}', End='{TRADING_END_TIME_STR}' (UTC)")
    run()